In [2]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from llama_index.core import Settings
from llama_index.core.callbacks import CallbackManager
from langfuse.llama_index import LlamaIndexCallbackHandler

langfuse_callback_handler = LlamaIndexCallbackHandler(
    public_key="pk-lf-29339a8f-8a05-4d10-9a0a-a4718f79a53d",
    secret_key="sk-lf-d1f94511-893b-456e-973e-f57b44d50a30",
    host="https://cloud.langfuse.com"
)
Settings.callback_manager = CallbackManager([langfuse_callback_handler])

#### Load Documents from JSON file

In [3]:
import json

with open('E-9_Visa_Docs.json', 'r') as f:
    visa_docs_json = json.load(f)

In [ ]:
""" Load Documents from JSON file """

from llama_index.core.schema import Document

visa_docs = [
    Document(
        text=doc["text"], 
        metadata=doc["metadata"], 
        excluded_llm_metadata_keys=doc["excluded_llm_metadata_keys"], 
        excluded_embed_metadata_keys=doc["excluded_embed_metadata_keys"],
    ) 
    for doc in visa_docs_json
]

In [4]:
""" Load TextNodes from JSON file """

from llama_index.core.schema import TextNode

visa_nodes = [
    TextNode(
        text=doc["text"],
        metadata=doc["metadata"],
        excluded_llm_metadata_keys=doc["excluded_llm_metadata_keys"],
        excluded_embed_metadata_keys=doc["excluded_embed_metadata_keys"],
    )
    for doc in visa_docs_json
]

#### Get VectorStore Index

In [32]:
import chromadb

db = chromadb.PersistentClient(path="./chroma_data")
chroma_collection = db.get_or_create_collection("E9_VISA")

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [43]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

# Base
base_index = VectorStoreIndex(visa_nodes, show_progress=True)
# Optional Persistent
# base_index.docstore.persist("base_data")



# Chroma
chroma_vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# From TextNodes
# Data Persistent 
# chroma_storage_context = StorageContext.from_defaults(vector_store=chroma_vector_store)
# chroma_index = VectorStoreIndex(visa_nodes, storage_context=chroma_storage_context, show_progress=True)

# From Documents
# chroma_index = VectorStoreIndex.from_documents(visa_docs, storage_context=chroma_storage_context)

# Load Data From Collection
chroma_index = VectorStoreIndex.from_vector_store(vector_store=chroma_vector_store)




# Qdrant
# client = QdrantClient("localhost", port="6333")
client = QdrantClient(path="./qdrant_data")
qdrant_vector_store = QdrantVectorStore(client=client, collection_name="E9_VISA_DOCS", enable_hybrid=True, batch_size=20)

# From TextNodes
# Data Persistent 
# qdrant_storage_context = StorageContext.from_defaults(vector_store=qdrant_vector_store)
# qdrant_index = VectorStoreIndex(visa_nodes, storage_context=qdrant_storage_context, show_progress=True)

# From Documents
# qdrant_index = VectorStoreIndex.from_documents(visa_docs, storage_context=qdrant_storage_context)


# Load Data From Collection
qdrant_index = VectorStoreIndex.from_vector_store(vector_store=qdrant_vector_store)



Generating embeddings:   0%|          | 0/21 [00:00<?, ?it/s]

In [98]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

reranker = CohereRerank(
    top_n=3, model="rerank-multilingual-v2.0"
)

# query_engine = index.as_query_engine(
#     streaming=True, 
#     similarity_top_k=20,
#     node_postprocessors=[reranker],
# )

### Set Retrievers

In [99]:
base_retriever = base_index.as_retriever(similarity_top_k=20, node_postprocessors=[reranker])
chroma_retriever = chroma_index.as_retriever(similarity_top_k=10)
# qdrant_retriever = qdrant_index.as_retriever(similarity_top_k=10, sparse_top_k=5, vector_store_query_mode="hybrid")
qdrant_retriever = qdrant_index.as_retriever(similarity_top_k=20, sparse_top_k=12, vector_store_query_mode="hybrid")

In [41]:
def pretty_print_source_nodes(source_nodes):
    for r in source_nodes:
        source = r.metadata['page_label']
        title = r.metadata['document_title']
        # text = r.text.split("\n")[0]
        print(f"Score: {r.score:.3f}   {source}: {title}")
        # print(f"{r.text[:50]}\n")

In [101]:
query = "저는 한국에 살고 있는 외국인인데, 직업을 바꾸는게 가능한가요?"
query = "한국에서 건물 청소하는 일을 할 예정인데 건물 청소는 어떤 업종이고 체류자격의 비자 타입은 무엇인가요?"
query = "현재 농장에서 일하고 있는데 이 분야가 계절적 특성을 타기 때문에 다음주 부턴 일이 없는데, 다른 분야의 사업장에 일을 구하는게 가능한가요?"
query = "올해부터 E9 비자로 식당에서 일을 할 수 있는 것 맞나요?"
query = "고용허가제를 통해 E9 비자 재입국 방법은 무엇입니까"
query = "E9 비자 외국인이 한국에 계속 체류하는 방법은 무엇입니까?"

print(f"Question: {query}\n")

print(f"---------------------------------------------Base Index---------------------------------------------------------")
langfuse_callback_handler.set_trace_params(
    name=query,
    version="1",
    tags=["Base Retriever", "Visa Guide Doc"]
)
source_nodes1 = base_retriever.retrieve(query)
pretty_print_source_nodes(source_nodes=source_nodes1)


# print(f"---------------------------------------------Chroma Index--------------------------------------------------------")
# source_nodes2 = chroma_retriever.retrieve(query)
# pretty_print_source_nodes(source_nodes=source_nodes2)

langfuse_callback_handler.set_trace_params(
    name=query,
    version="2",
    tags=["Qdrant Retriever", "Visa Guide Doc"]
)
print(f"---------------------------------------------Qdrant Index---------------------------------------------------------")
source_nodes3 = qdrant_retriever.retrieve(query)
pretty_print_source_nodes(source_nodes=source_nodes3)

Question: E9 비자 외국인이 한국에 계속 체류하는 방법은 무엇입니까?

---------------------------------------------Base Index---------------------------------------------------------
Score: 0.855   5: E-9 근로자를 채용할 수 있는 업종과 범위
Score: 0.847   21: 고용허가제 해당자의 고용변동 신고
Score: 0.841   1: E-9 Visa Guide: 목차
Score: 0.829   17: 고용허가제 해당자의 체류자격 변경허가 - 사증 변경
Score: 0.826   6: 고용허가제 해당자는 사증발급인정서를 받아야만 E-9 사증을 발급받을 수 있음
Score: 0.817   20: 고용허가제 해당자의 외국인등록
Score: 0.815   8: 비전문취업(E-9) 자격 사증발급인정서 발급절차
Score: 0.814   10: 재입국특례자에 대한 사증 신청 및 발급 방법
Score: 0.812   19: 고용허가제 해당자의 재입국허가
Score: 0.811   2: 고용허가제 법률적 개요와 고용허가제 선정국가
Score: 0.810   4: 고용허가제 해당자의 사증발급 후 1회 체류기간의 상한
Score: 0.809   12: 고용허가제 해당자의 체류자격 외 활동
Score: 0.809   11: 재입국특례자에 대한 우대 내용
Score: 0.806   9: 재입국특례 제도(구 성실근로자 제도)
Score: 0.806   18: 고용허가제 해당자의 체류기간 연장 허가 - 사증 연장
Score: 0.805   3: 고용허가제 해당자와 활동범위
Score: 0.798   16: 고용허가제 농업 분야 외국인근로자의 근무처(직장) 추가
Score: 0.793   14: 고용허가제 해당자가 근무처(직장)를 변경할 수 있는 조건
Score: 0.784   7: 고용허가제 해당자는 2012.8.1부터 범죄경력증명서 및 건강상태확인서 제출
Sco

In [55]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4-1106-preview", temperature=0)

In [65]:
base_engine = base_index.as_query_engine()
chroma_engine = chroma_index.as_query_engine()
qdrant_engine = qdrant_index.as_query_engine(similarity_top_k=10, sparse_top_k=5, vector_store_query_mode="hybrid")

In [50]:
def llm_query(query: str):
    # langfuse_callback_handler.set_trace_params(
    #     name=query,
    #     version="1",
    #     tags=["Visa Guide Doc", "BaseEngine"]
    # )
    # response = base_engine.query(query)
    # print(f"BaseEngine: {response}\n\n")

    langfuse_callback_handler.set_trace_params(
        name=query,
        version="2",
        tags=["Visa Guide Doc", "ChromaEngine"]
    )
    response = chroma_engine.query(query)
    print(f"ChromaEngine: {response}\n\n")

    langfuse_callback_handler.set_trace_params(
        name=query,
        version="3",
        tags=["Visa Guide Doc", "QdrantEngine"]
    )
    response = qdrant_engine.query(query)
    print(f"QdrantEngine: {response}\n\n")


In [66]:
query = "저는 한국에 살고 있는 외국인인데, 직업을 바꾸는게 가능한가요?"
query = "한국에서 건물 청소하는 일을 할 예정인데 건물 청소는 어떤 업종이고 체류자격의 비자 타입은 무엇인가요?"
# query = "현재 농장에서 일하고 있는데 이 분야가 계절적 특성을 타기 때문에 다음주 부턴 일이 없는데, 다른 분야의 사업장에 일을 구하는게 가능한가요?"
# query = "올해부터 E9 비자로 식당에서 일을 할 수 있는 것 맞나요?"

llm_query(query)

ChromaEngine: 건물 청소하는 일은 서비스업에 속하며, 해당 체류자격의 비자 타입은 E-9-5입니다.


QdrantEngine: 건물 청소는 서비스업에 속하며, 해당 체류자격의 비자 타입은 E-9-5입니다.


